In [2]:
import tensorflow as tf
import numpy as np
import os

In [3]:
label_decoder = {
    "en": 0,
    "es": 1,
    "de": 2
}

In [4]:
def one_hot_encode(i):
    label_one_hot = [0, 0, 0]
    label_one_hot[i] = 1
    return label_one_hot

In [5]:
image_directory = os.listdir('./dataset/train_spectrogram_images')
image_labels = image_directory
image_directory = ['dataset/train_spectrogram_images/' + i for i in image_directory]
images = []
labels = []
for i in image_directory:
    flag_images_dir = [(i+'/'+j) for j in os.listdir(i)]
    flag_labels = [(label_decoder[i[-2:]]) for j in os.listdir(i)]
    flag_labels = [one_hot_encode(i) for i in flag_labels]
    labels = labels + flag_labels
    images = images + flag_images_dir

In [6]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [432, 432])
    image /= 255.0
    return image

In [7]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [8]:
BATCH_SIZE = 16
AUTOTUNE = 1
image_count = len(images)

In [9]:
ds = tf.data.Dataset.from_tensor_slices((images, labels))

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
ds = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
iterator = ds.make_one_shot_iterator()

In [10]:
n_classes = 3
x = tf.placeholder('float', [None, 432, 432, 3])
y = tf.placeholder('float', [None, n_classes])

In [11]:
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [1]:
def convolutional_neural_network(x):
    weights = {
        'w_conv1':tf.Variable(tf.random_normal([5,5,3,32])),
        'w_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
        'w_conv3':tf.Variable(tf.random_normal([5,5,64,128])),
        'w_fc':tf.Variable(tf.random_normal([54*54*128,1000])),
        'out':tf.Variable(tf.random_normal([1000, n_classes]))
    }

    biases = {
        'b_conv1':tf.Variable(tf.random_normal([32])),
        'b_conv2':tf.Variable(tf.random_normal([64])),
        'b_conv3':tf.Variable(tf.random_normal([128])),
        'b_fc':tf.Variable(tf.random_normal([1000])),
        'out':tf.Variable(tf.random_normal([n_classes]))
    }

    conv1 = tf.nn.relu(conv2d(x, weights['w_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    conv2 = tf.nn.relu(conv2d(conv1, weights['w_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    conv3 = tf.nn.relu(conv2d(conv2, weights['w_conv3']) + biases['b_conv3'])
    conv3 = maxpool2d(conv3)
    fc = tf.reshape(conv3,[-1, 54*54*128])
    fc = tf.matmul(fc, weights['w_fc'])+biases['b_fc']
    output = tf.matmul(fc, weights['out'])+biases['out']
    return output

In [ ]:
def train_neural_network(x, epochs = 10):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = prediction) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(epochs):
            epoch_loss = 0
            for i in range(int(len(images)/BATCH_SIZE)):
                image_batch, label_batch = iterator.get_next()
                image_batch, label_batch = sess.run([image_batch, label_batch])
                _, c = sess.run([optimizer, cost], feed_dict={
                    x: np.array(image_batch), 
                    y: np.array(label_batch)
                })
                epoch_loss += c
                print('Batch:', i, 'completed of', BATCH_SIZE)
            print('Epoch', epoch, 'completed out of', epochs, 'loss:', epoch_loss)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

train_neural_network(x)